In [1]:
import os
import sys
os.chdir('../')
sys.path.append(os.path.join(os.getcwd(), "src"))

In [2]:
from dataclasses import dataclass
from pathlib import Path
from electron.utils.helpers import *
from electron.utils.exception import *
from electron.constants import *
from electron import logger

In [3]:
@dataclass
class ModelEvaluationConfig:
    model_path: Path
    x_transform: Path
    y_transform: Path
    metrics_path: Path

In [4]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_PATH,
                 params_filepath = PARAMS_PATH,
                 schema_filepath = SCHEMA_PATH):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        model_evaluation_config =  ModelEvaluationConfig(
            model_path=Path(config.model_path),
            x_transform=Path(config.x_transform),
            y_transform=Path(config.y_transform),
            metrics_path=Path(config.metrics_path)
        )

        return model_evaluation_config

In [5]:
import os
import json
import joblib
import mlflow
import dagshub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


class ModelEvaluation:
    def __init__(self, config):
        self.config = config
        mlflow.set_tracking_uri("file:./mlruns")
        mlflow.set_experiment("Electricity Demand Prediction")
        logger.info("MLflow tracking setup complete.")

    def evaluate(self):
        try:
            # Load model and test data
            model = joblib.load(self.config.model_path)
            x_test = np.load(self.config.x_transform, allow_pickle=True)
            y_test = np.load(self.config.y_transform, allow_pickle=True).squeeze()

            # Predict
            preds = model.predict(x_test)

            # Metrics
            metrics = {
                "mse": mean_squared_error(y_test, preds),
                "mae": mean_absolute_error(y_test, preds),
                "rmse": np.sqrt(mean_squared_error(y_test, preds)),
                "mape": np.mean(np.abs((y_test - preds) / y_test)) * 100 if np.any(y_test != 0) else np.inf,
                "r2_score": r2_score(y_test, preds),
                "adjusted_r2": 1 - (1 - r2_score(y_test, preds)) * (len(y_test) - 1) / (len(y_test) - x_test.shape[1] - 1)
            }

            create_directories([Path(self.config.metrics_path).parent])
            save_json(Path(self.config.metrics_path), metrics)
            logger.info(f"Model evaluation metrics: {metrics}")
            # Log to MLflow
            with mlflow.start_run(run_name="Model Evaluation"):
                mlflow.log_metrics({k: float(v) for k, v in metrics.items()})
                mlflow.set_tag("stage", "evaluation")
                mlflow.log_artifact(self.config.metrics_path)
                mlflow.log_artifact(self.config.model_path)

            logger.info("✅ Model evaluation complete. Metrics logged.")
            return metrics

        except Exception as e:
            raise CustomException(e, sys)


In [6]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise CustomException(str(e), sys)

[2025-07-06 11:15:05,012: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-06 11:15:05,012: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]


[2025-07-06 11:15:05,029: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-07-06 11:15:05,031: INFO: helpers: created directory at: artifacts]
[2025-07-06 11:15:05,060: INFO: 2097186014: MLflow tracking setup complete.]


c:\Users\Javith Naseem\.conda\envs\ele\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[2025-07-06 11:15:09,555: INFO: helpers: created directory at: artifacts\model_evaluation]
[2025-07-06 11:15:09,555: INFO: helpers: json file saved at: artifacts\model_evaluation\metrics.json]
[2025-07-06 11:15:09,555: INFO: 2097186014: Model evaluation metrics: {'mse': 581.2932073192122, 'mae': 16.75716600136802, 'rmse': 24.110022963888113, 'mape': 1.4316218193159305, 'r2_score': 0.9997356598514628, 'adjusted_r2': 0.9997331910601854}]
[2025-07-06 11:15:09,694: INFO: 2097186014: ✅ Model evaluation complete. Metrics logged.]
